In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
import hopsworks

# Login to the Hopsworks project using your API key
project = hopsworks.login(
    api_key_value="mAaHAWHefj9ff3oE.OgcnSgFgldipqzmX8pbMMnkM2sEY2SpIeGVrrcuIlNwwM5I89Oe3jZj2C8CE8oTm")

# Get the feature store instance
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1047702
2024-10-06 13:27:37,833 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-10-06 13:27:37,834 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_group_city_weather = fs.get_feature_group(
    name="city_weather_features", version=1)
feature_group_trucks = fs.get_feature_group(name="trucks_features", version=1)
feature_group_truck_schedule = fs.get_feature_group(
    name="trucks_schedule_features", version=1)
feature_group_routes_weather = fs.get_feature_group(
    name="routes_weather_features", version=1)
feature_group_routes = fs.get_feature_group(name="routes_features", version=1)
feature_group_drivers = fs.get_feature_group(
    name='drivers_features', version=1)
feature_group_traffic = fs.get_feature_group(
    name="traffic_table_features", version=1)


In [4]:
df_trucks = feature_group_trucks.read()
df_truck_schedule = feature_group_truck_schedule.read()
df_traffic = feature_group_traffic.read()
df_routes_weather = feature_group_routes_weather.read()
df_routes = feature_group_routes.read()

df_drivers = feature_group_drivers.read()
df_city_weather = feature_group_city_weather.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.71s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.27s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.76s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.45s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.50s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.44s) 


In [5]:
df_drivers.shape

(1283, 11)

In [6]:
df_drivers.isnull().sum()

index                0
driver_id            0
name                 0
gender               0
age                  0
experience           0
driving_style        0
ratings              0
vehicle_no           0
average_speed_mph    0
event_date           0
dtype: int64

In [7]:
df_routes.isnull().sum()

index             0
route_id          0
origin_id         0
destination_id    0
distance          0
average_hours     0
event_date        0
dtype: int64

In [8]:
df_traffic.shape

(2597913, 6)

In [9]:
df_traffic.isnull().sum()

index             0
route_id          0
no_of_vehicles    0
accident          0
datetime          0
event_date        0
dtype: int64

In [10]:
df_routes.shape

(1810, 7)

In [11]:
df_routes_weather.shape

(396149, 11)

In [12]:
df_city_weather.shape

(51834, 11)

In [13]:
df_trucks.shape

(1296, 7)

In [14]:
df_truck_schedule.shape

(12308, 7)

In [15]:
df_drivers.shape

(1283, 11)

In [16]:
df_truck_schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   index              12308 non-null  int32              
 1   truck_id           12308 non-null  int32              
 2   route_id           12308 non-null  object             
 3   departure_date     12308 non-null  object             
 4   estimated_arrival  12308 non-null  datetime64[us, UTC]
 5   delay              12308 non-null  int32              
 6   event_date         12308 non-null  datetime64[us, UTC]
dtypes: datetime64[us, UTC](2), int32(3), object(2)
memory usage: 529.0+ KB


In [17]:
df_truck_schedule

,index,truck_id,route_id,departure_date,estimated_arrival,delay,event_date
0,6656,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36+00:00,0,2024-10-03 00:00:00+00:00
1,10174,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00+00:00,1,2024-10-03 00:00:00+00:00
2,2883,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36+00:00,0,2024-10-03 00:00:00+00:00
3,5313,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23.999000+00:00,0,2024-10-03 00:00:00+00:00
4,960,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12+00:00,0,2024-10-03 00:00:00+00:00
...,...,...,...,...,...,...,...
12303,8987,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00+00:00,1,2024-10-03 00:00:00+00:00
12304,11561,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24+00:00,0,2024-10-03 00:00:00+00:00
12305,1964,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36+00:00,1,2024-10-03 00:00:00+00:00
12306,7287,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48+00:00,0,2024-10-03 00:00:00+00:00


In [18]:
from datetime import datetime
import pandas as pd
df_city_weather['datetime'] = df_city_weather['datetime'].astype(str)

# Now you can use the .str accessor
df_city_weather['datetime'] = df_city_weather['datetime'].str.split('+').str[0]

# Optionally convert back to datetime
df_city_weather['datetime'] = pd.to_datetime(df_city_weather['datetime'])

In [19]:
# Step 1: Remove the part after the '+' (including any fractional seconds)
df_truck_schedule['estimated_arrival'] = df_truck_schedule['estimated_arrival'].astype(
    str).str.split('+').str[0].str.split('.').str[0]

# Step 2: Convert to datetime without the fractional seconds
df_truck_schedule['estimated_arrival'] = pd.to_datetime(
    df_truck_schedule['estimated_arrival'], format='%Y-%m-%d %H:%M:%S')

# Display the result
df_truck_schedule

,index,truck_id,route_id,departure_date,estimated_arrival,delay,event_date
0,6656,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2024-10-03 00:00:00+00:00
1,10174,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2024-10-03 00:00:00+00:00
2,2883,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2024-10-03 00:00:00+00:00
3,5313,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2024-10-03 00:00:00+00:00
4,960,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2024-10-03 00:00:00+00:00
...,...,...,...,...,...,...,...
12303,8987,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2024-10-03 00:00:00+00:00
12304,11561,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,2024-10-03 00:00:00+00:00
12305,1964,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2024-10-03 00:00:00+00:00
12306,7287,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2024-10-03 00:00:00+00:00


In [20]:
df_truck_schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12308 entries, 0 to 12307
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   index              12308 non-null  int32              
 1   truck_id           12308 non-null  int32              
 2   route_id           12308 non-null  object             
 3   departure_date     12308 non-null  object             
 4   estimated_arrival  12308 non-null  datetime64[ns]     
 5   delay              12308 non-null  int32              
 6   event_date         12308 non-null  datetime64[us, UTC]
dtypes: datetime64[ns](1), datetime64[us, UTC](1), int32(3), object(2)
memory usage: 529.0+ KB


In [21]:
df_traffic

,index,route_id,no_of_vehicles,accident,datetime,event_date
0,1693789,R-2278e851,522.0,0,2019-01-03 04:00:00+00:00,2024-10-06 00:00:00+00:00
1,2082177,R-0e3ff935,216.0,0,2019-02-09 00:00:00+00:00,2024-10-06 00:00:00+00:00
2,776525,R-4c68ff74,425.0,1,2019-01-18 05:00:00+00:00,2024-10-06 00:00:00+00:00
3,1892953,R-2d8055b0,2602.0,0,2019-01-21 16:00:00+00:00,2024-10-06 00:00:00+00:00
4,1803755,R-a08baf8e,565.0,0,2019-01-31 02:00:00+00:00,2024-10-06 00:00:00+00:00
...,...,...,...,...,...,...
2597908,157086,R-3cad4a95,684.0,0,2019-01-14 06:00:00+00:00,2024-10-06 00:00:00+00:00
2597909,442011,R-ad451391,828.0,1,2019-01-18 03:00:00+00:00,2024-10-06 00:00:00+00:00
2597910,1881620,R-0685acb5,1900.0,0,2019-01-09 11:00:00+00:00,2024-10-06 00:00:00+00:00
2597911,2015894,R-fd52f991,780.0,0,2019-02-07 05:00:00+00:00,2024-10-06 00:00:00+00:00


In [22]:
from datetime import datetime
df_traffic['datetime'] = df_traffic['datetime'].astype(str)

# Now you can use the .str accessor
df_traffic['datetime'] = df_traffic['datetime'].str.split('+').str[0]

# Optionally convert back to datetime
df_traffic['datetime'] = pd.to_datetime(df_traffic['datetime'])

In [23]:
df_trucks = df_trucks.drop(columns=['index', 'event_date'])
df_truck_schedule = df_truck_schedule.drop(columns=['index', 'event_date'])
df_traffic = df_traffic.drop(columns=['index', 'event_date'])
df_routes_weather = df_routes_weather.drop(columns=['index', 'event_date'])
df_routes = df_routes.drop(columns=['index', 'event_date'])
df_drivers = df_drivers.drop(columns=['index', 'event_date'])
df_city_weather = df_city_weather.drop(columns=['index', 'event_date'])

In [24]:
# origin_weather = pd.merge(df_routes, df_city_weather, how='left', left_on='origin_id', right_on='city_id', suffixes=('', '_origin'))

# # Merging for destination city weather
# weather_info = pd.merge(origin_weather, df_city_weather, how='left', left_on='destination_id', right_on='city_id', suffixes=('', '_destination'))

# # Selecting relevant columns

In [25]:
df_truck_schedule

,truck_id,route_id,departure_date,estimated_arrival,delay
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0
...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0


In [26]:
df_routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-57a59d1d,2019-01-31 06:00:00,79,12,Sunny,0.0,59,6,1014
1,R-85d93d3b,2019-02-13 06:00:00,63,12,Sunny,0.0,60,6,1018
2,R-af1d28da,2019-02-03 06:00:00,63,7,Clear,0.0,72,6,1014
3,R-86eb239c,2019-01-30 12:00:00,59,6,Light drizzle,0.0,81,3,1015
4,R-7a0a08ef,2019-02-08 18:00:00,45,2,Sunny,0.0,76,6,1019
...,...,...,...,...,...,...,...,...,...
396144,R-b2334f87,2019-02-10 00:00:00,7,14,Sunny,0.0,51,6,1028
396145,R-ad9044ec,2019-01-29 00:00:00,73,8,Sunny,0.0,62,6,1019
396146,R-47c6c141,2019-01-01 06:00:00,63,4,Light rain shower,0.0,76,6,1007
396147,R-6b656856,2019-02-09 00:00:00,30,14,Sunny,0.0,64,6,1030


In [27]:
df_drivers

,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,73abe416-e,David Kennedy,male,48,12.13354,conservative,8,24034677,42.669998
1,489ecdc6-9,Robert Navarro,male,51,30.00000,proactive,6,38275836,58.500000
2,baf19ca4-4,Craig Harmon,male,49,16.00000,conservative,9,26687586,49.270000
3,210ca93c-0,Christopher Weaver,male,59,16.00000,conservative,6,23471751,48.419998
4,eda4cbe3-3,Matthew Soto,male,52,26.00000,conservative,4,85416403,42.639999
...,...,...,...,...,...,...,...,...,...
1278,ef5c69e8-5,Travis Lawrence,male,52,25.00000,conservative,7,98052326,48.110001
1279,1af7f86d-f,Anthony Wolfe,male,54,24.00000,proactive,9,21097795,57.860001
1280,3ec1b4cb-3,Richard Delacruz,male,48,14.00000,proactive,5,16430282,61.610001
1281,df945cf8-d,Richard Harris,male,51,17.00000,proactive,4,25666034,61.880001


In [28]:
df_trucks

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,29303248,9,4000.0,15,gas
1,29292035,9,6000.0,24,diesel
2,17287024,12,15000.0,17,gas
3,11098873,11,15000.0,20,gas
4,22171671,8,20000.0,25,diesel
...,...,...,...,...,...
1291,67906530,11,6000.0,22,diesel
1292,11072962,7,15000.0,20,diesel
1293,29780011,15,4000.0,19,gas
1294,32628685,11,15000.0,18,gas


In [29]:
df_city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-328bd8d3,14,16,Heavy snow,0.0,82,1,1019,2019-01-25 01:00:00
1,C-841ebdcb,54,6,Sunny,0.0,44,6,1023,2019-01-24 17:00:00
2,C-4fe0fa24,32,2,Patchy heavy snow,0.0,80,2,1015,2019-01-09 02:00:00
3,C-03bb3e48,-4,12,Partly cloudy,0.0,89,6,1023,2019-02-14 21:00:00
4,C-6df8beaf,36,13,Light sleet,0.0,95,6,1001,2019-01-28 05:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-90e30162,66,6,Light rain shower,0.0,95,6,1017,2019-02-05 21:00:00
51830,C-01660979,34,5,Light snow,0.0,47,6,1029,2019-02-12 00:00:00
51831,C-19236709,27,2,Overcast,0.0,85,6,1022,2019-02-01 03:00:00
51832,C-b5282c3b,9,18,Blizzard,0.0,75,0,1011,2019-01-29 14:00:00


In [30]:
df_city_weather.drop_duplicates(subset=['city_id', 'datetime'], inplace=True)
df_routes_weather.drop_duplicates(subset=['route_id', 'date'], inplace=True)
df_trucks.drop_duplicates(subset=['truck_id'], inplace=True)
df_drivers.drop_duplicates(subset=['driver_id'], inplace=True)
df_routes.drop_duplicates(
    subset=['route_id', 'destination_id', 'origin_id'], inplace=True)
df_truck_schedule.drop_duplicates(
    subset=['truck_id', 'route_id', 'departure_date'], inplace=True)

In [31]:
df_city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-328bd8d3,14,16,Heavy snow,0.0,82,1,1019,2019-01-25 01:00:00
1,C-841ebdcb,54,6,Sunny,0.0,44,6,1023,2019-01-24 17:00:00
2,C-4fe0fa24,32,2,Patchy heavy snow,0.0,80,2,1015,2019-01-09 02:00:00
3,C-03bb3e48,-4,12,Partly cloudy,0.0,89,6,1023,2019-02-14 21:00:00
4,C-6df8beaf,36,13,Light sleet,0.0,95,6,1001,2019-01-28 05:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-90e30162,66,6,Light rain shower,0.0,95,6,1017,2019-02-05 21:00:00
51830,C-01660979,34,5,Light snow,0.0,47,6,1029,2019-02-12 00:00:00
51831,C-19236709,27,2,Overcast,0.0,85,6,1022,2019-02-01 03:00:00
51832,C-b5282c3b,9,18,Blizzard,0.0,75,0,1011,2019-01-29 14:00:00


In [32]:
df_routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-57a59d1d,2019-01-31 06:00:00,79,12,Sunny,0.0,59,6,1014
1,R-85d93d3b,2019-02-13 06:00:00,63,12,Sunny,0.0,60,6,1018
2,R-af1d28da,2019-02-03 06:00:00,63,7,Clear,0.0,72,6,1014
3,R-86eb239c,2019-01-30 12:00:00,59,6,Light drizzle,0.0,81,3,1015
4,R-7a0a08ef,2019-02-08 18:00:00,45,2,Sunny,0.0,76,6,1019
...,...,...,...,...,...,...,...,...,...
396144,R-b2334f87,2019-02-10 00:00:00,7,14,Sunny,0.0,51,6,1028
396145,R-ad9044ec,2019-01-29 00:00:00,73,8,Sunny,0.0,62,6,1019
396146,R-47c6c141,2019-01-01 06:00:00,63,4,Light rain shower,0.0,76,6,1007
396147,R-6b656856,2019-02-09 00:00:00,30,14,Sunny,0.0,64,6,1030


In [33]:
import pandas as pd

# Assuming df_truck_schedule and df_routes_weather are already loaded and datetime columns are properly formatted
df_truck_schedule['estimated_arrival'] = pd.to_datetime(
    df_truck_schedule['estimated_arrival'])
df_truck_schedule['departure_date'] = pd.to_datetime(
    df_truck_schedule['departure_date'])
df_routes_weather['date'] = pd.to_datetime(df_routes_weather['date'])

# Round the estimated_arrival to the nearest upper 6 hours and departure_date to the nearest lower 6 hours
df_truck_schedule1 = df_truck_schedule.copy()

In [34]:
df_truck_schedule

,truck_id,route_id,departure_date,estimated_arrival,delay
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0
...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0


In [35]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0
...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0


In [36]:
df_truck_schedule1['estimated_arrival'] = df_truck_schedule1['estimated_arrival'].dt.ceil(
    '6H')
df_truck_schedule1['departure_date'] = df_truck_schedule1['departure_date'].dt.floor(
    '6H')

In [37]:
df_truck_schedule1['date'] = [
    pd.date_range(start=row['departure_date'],
                  end=row['estimated_arrival'], freq='6H')
    for _, row in df_truck_schedule1.iterrows()
]

In [38]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,"DatetimeIndex(['2019-01-31 06:00:00', '2019-01..."
1,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,"DatetimeIndex(['2019-01-26 06:00:00', '2019-01..."
2,29077811,R-577e285c,2019-01-13 06:00:00,2019-01-13 12:00:00,0,"DatetimeIndex(['2019-01-13 06:00:00', '2019-01..."
3,12492275,R-a6df9ba2,2019-01-25 06:00:00,2019-01-25 18:00:00,0,"DatetimeIndex(['2019-01-25 06:00:00', '2019-01..."
4,10784487,R-c970877c,2019-01-04 06:00:00,2019-01-04 12:00:00,0,"DatetimeIndex(['2019-01-04 06:00:00', '2019-01..."
...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 06:00:00,2019-02-12 18:00:00,1,"DatetimeIndex(['2019-02-12 06:00:00', '2019-02..."
12304,74796611,R-c084fe21,2019-01-01 06:00:00,2019-01-07 06:00:00,0,"DatetimeIndex(['2019-01-01 06:00:00', '2019-01..."
12305,25670063,R-9d0d552b,2019-01-10 06:00:00,2019-01-11 00:00:00,1,"DatetimeIndex(['2019-01-10 06:00:00', '2019-01..."
12306,42302347,R-269c91ef,2019-02-03 06:00:00,2019-02-03 18:00:00,0,"DatetimeIndex(['2019-02-03 06:00:00', '2019-02..."


In [39]:
df_truck_schedule1 = df_truck_schedule1.explode('date')

In [40]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00
1,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00
...,...,...,...,...,...,...
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00


In [41]:
df_truck_schedule1 = df_truck_schedule1.drop_duplicates()

In [42]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00
1,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00
...,...,...,...,...,...,...
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00


In [43]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00
1,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00
...,...,...,...,...,...,...
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00


In [44]:


# Now merge the dataframes based on 'route_id' and the rounded dates
# Merging on route_id and matching the departure date with the weather date for that route
df_merged = pd.merge(
    df_truck_schedule1,
    df_routes_weather,
    on=['route_id', 'date'],
    how='left'  # Using left join to retain all records from truck schedule even if no weather data is found
)

# Drop the 'date' column from weather data as it's redundant now
# df_merged.drop(columns=['date'], inplace=True)

# The result is a merged dataframe with truck schedules and corresponding route weather
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00,30.0,12.0,Cloudy,0.0,83.0,6.0,1025.0
1,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00,64.0,6.0,Clear,0.0,69.0,6.0,1018.0
2,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00,63.0,7.0,Clear,0.0,67.0,6.0,1019.0
3,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00,32.0,14.0,Cloudy,0.0,79.0,6.0,1025.0
4,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00,34.0,6.0,Fog,0.0,99.0,0.0,1007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71371,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00,63.0,1.0,Patchy rain possible,0.0,86.0,6.0,1015.0
71372,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00,28.0,19.0,Patchy light snow,0.0,90.0,6.0,1022.0
71373,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00,28.0,20.0,Patchy light snow,0.0,88.0,6.0,1023.0
71374,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00,70.0,1.0,Patchy rain possible,0.0,82.0,6.0,1015.0


In [45]:
df_merged.isnull().sum()

truck_id                0
route_id                0
departure_date          0
estimated_arrival       0
delay                   0
date                    0
temp                 5199
wind_speed           5199
description          5199
precip               5199
humidity             5199
visibility           5199
pressure             5199
dtype: int64

In [46]:
df_merged = df_merged.dropna(
    subset=['temp', 'wind_speed', 'description', 'humidity', 'pressure'])
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00,30.0,12.0,Cloudy,0.0,83.0,6.0,1025.0
1,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00,64.0,6.0,Clear,0.0,69.0,6.0,1018.0
2,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00,63.0,7.0,Clear,0.0,67.0,6.0,1019.0
3,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00,32.0,14.0,Cloudy,0.0,79.0,6.0,1025.0
4,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00,34.0,6.0,Fog,0.0,99.0,0.0,1007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71371,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00,63.0,1.0,Patchy rain possible,0.0,86.0,6.0,1015.0
71372,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00,28.0,19.0,Patchy light snow,0.0,90.0,6.0,1022.0
71373,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00,28.0,20.0,Patchy light snow,0.0,88.0,6.0,1023.0
71374,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00,70.0,1.0,Patchy rain possible,0.0,82.0,6.0,1015.0


In [47]:
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00,30.0,12.0,Cloudy,0.0,83.0,6.0,1025.0
1,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00,64.0,6.0,Clear,0.0,69.0,6.0,1018.0
2,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00,63.0,7.0,Clear,0.0,67.0,6.0,1019.0
3,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00,32.0,14.0,Cloudy,0.0,79.0,6.0,1025.0
4,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00,34.0,6.0,Fog,0.0,99.0,0.0,1007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71371,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00,63.0,1.0,Patchy rain possible,0.0,86.0,6.0,1015.0
71372,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00,28.0,19.0,Patchy light snow,0.0,90.0,6.0,1022.0
71373,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00,28.0,20.0,Patchy light snow,0.0,88.0,6.0,1023.0
71374,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00,70.0,1.0,Patchy rain possible,0.0,82.0,6.0,1015.0


In [48]:
df_merged.isnull().sum()

truck_id             0
route_id             0
departure_date       0
estimated_arrival    0
delay                0
date                 0
temp                 0
wind_speed           0
description          0
precip               0
humidity             0
visibility           0
pressure             0
dtype: int64

In [49]:
df_merged.shape

(66177, 13)

In [50]:
def custom_mode(x):
    """Calculate the mode of a pandas Series."""
    return x.mode().iloc[0]

In [51]:
df_merged

,truck_id,route_id,departure_date,estimated_arrival,delay,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00,30.0,12.0,Cloudy,0.0,83.0,6.0,1025.0
1,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00,64.0,6.0,Clear,0.0,69.0,6.0,1018.0
2,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00,63.0,7.0,Clear,0.0,67.0,6.0,1019.0
3,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00,32.0,14.0,Cloudy,0.0,79.0,6.0,1025.0
4,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00,34.0,6.0,Fog,0.0,99.0,0.0,1007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71371,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00,63.0,1.0,Patchy rain possible,0.0,86.0,6.0,1015.0
71372,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00,28.0,19.0,Patchy light snow,0.0,90.0,6.0,1022.0
71373,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00,28.0,20.0,Patchy light snow,0.0,88.0,6.0,1023.0
71374,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00,70.0,1.0,Patchy rain possible,0.0,82.0,6.0,1015.0


In [52]:
scheduled_weather = pd.DataFrame(df_merged)
schedule_weather_grp = scheduled_weather.groupby(['truck_id', 'route_id'], as_index=False).agg(
    route_avg_temp=('temp', 'mean'),
    route_avg_wind_speed=('wind_speed', 'mean'),

    route_avg_humidity=('humidity', 'mean'),

    route_avg_pressure=('pressure', 'mean'),
    route_avg_precip=('precip', 'mean'),
    route_avg_visibility=('visibility', 'mean'),
    route_description=('description', custom_mode)
)

# Display the aggregated DataFrame
schedule_weather_grp

,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,10008392,R-06dfe39e,50.750000,6.416667,79.416667,1015.250000,0.00,5.833333,Partly cloudy
1,10008392,R-35dca729,56.600000,9.400000,72.200000,1013.300000,0.00,5.700000,Partly cloudy
2,10008392,R-51d1a32a,46.636364,8.909091,76.181818,1016.818182,0.00,6.000000,Partly cloudy
3,10008392,R-7ab5ae85,58.750000,7.000000,82.250000,1016.000000,0.00,4.000000,Sunny
4,10008392,R-c6f8e036,47.916667,6.166667,88.500000,1011.000000,0.00,5.583333,Clear
...,...,...,...,...,...,...,...,...,...
12192,99981667,R-abbf83d1,59.000000,7.500000,80.750000,1014.500000,0.00,5.500000,Light rain shower
12193,99981667,R-e9bf94ff,58.400000,3.000000,70.800000,1016.400000,0.00,4.400000,Sunny
12194,99981667,R-ed1094fd,64.500000,10.500000,42.500000,1016.750000,0.00,6.000000,Clear
12195,99981667,R-f363ad37,79.000000,11.800000,84.800000,1013.600000,0.02,4.800000,Clear


In [53]:
# schedule_weather_grp = schedule_weather_grp.rename(
#     columns={'index_x': 'index'})

In [54]:
schedule_weather_merge = df_truck_schedule.merge(
    schedule_weather_grp,
    on=['truck_id', 'route_id'],
    how='left'
)

# Display the merged DataFrame
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,6.000000,Clear
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,3.000000,Fog
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,6.000000,Cloudy
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,6.000000,Sunny
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,4.500000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,4.000000,Heavy rain
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,5.440000,Clear
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,6.000000,Clear
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,4.333333,Clear


In [55]:
schedule_weather_merge.isnull().sum()

truck_id                 0
route_id                 0
departure_date           0
estimated_arrival        0
delay                    0
route_avg_temp          43
route_avg_wind_speed    43
route_avg_humidity      43
route_avg_pressure      43
route_avg_precip        43
route_avg_visibility    43
route_description       43
dtype: int64

In [56]:
schedule_weather_merge = schedule_weather_merge.dropna()
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,6.000000,Clear
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,3.000000,Fog
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,6.000000,Cloudy
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,6.000000,Sunny
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,4.500000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,4.000000,Heavy rain
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,5.440000,Clear
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,6.000000,Clear
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,4.333333,Clear


In [57]:
# schedule_weather_merge = schedule_weather_merge.drop(columns='date')

In [58]:
schedule_weather_merge = schedule_weather_merge.drop_duplicates()
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,6.000000,Clear
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,3.000000,Fog
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,6.000000,Cloudy
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,6.000000,Sunny
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,4.500000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,4.000000,Heavy rain
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,5.440000,Clear
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,6.000000,Clear
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,4.333333,Clear


In [59]:
schedule_weather_merge.isnull().any()

truck_id                False
route_id                False
departure_date          False
estimated_arrival       False
delay                   False
route_avg_temp          False
route_avg_wind_speed    False
route_avg_humidity      False
route_avg_pressure      False
route_avg_precip        False
route_avg_visibility    False
route_description       False
dtype: bool

In [60]:
schedule_weather_merge = schedule_weather_merge.drop_duplicates()

In [61]:
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,6.000000,Clear
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,3.000000,Fog
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,6.000000,Cloudy
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,6.000000,Sunny
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,4.500000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,4.000000,Heavy rain
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,5.440000,Clear
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,6.000000,Clear
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,4.333333,Clear


In [62]:
df_city_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-328bd8d3,14,16,Heavy snow,0.0,82,1,1019,2019-01-25 01:00:00
1,C-841ebdcb,54,6,Sunny,0.0,44,6,1023,2019-01-24 17:00:00
2,C-4fe0fa24,32,2,Patchy heavy snow,0.0,80,2,1015,2019-01-09 02:00:00
3,C-03bb3e48,-4,12,Partly cloudy,0.0,89,6,1023,2019-02-14 21:00:00
4,C-6df8beaf,36,13,Light sleet,0.0,95,6,1001,2019-01-28 05:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-90e30162,66,6,Light rain shower,0.0,95,6,1017,2019-02-05 21:00:00
51830,C-01660979,34,5,Light snow,0.0,47,6,1029,2019-02-12 00:00:00
51831,C-19236709,27,2,Overcast,0.0,85,6,1022,2019-02-01 03:00:00
51832,C-b5282c3b,9,18,Blizzard,0.0,75,0,1011,2019-01-29 14:00:00


In [63]:
df_routes_weather

,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
0,R-57a59d1d,2019-01-31 06:00:00,79,12,Sunny,0.0,59,6,1014
1,R-85d93d3b,2019-02-13 06:00:00,63,12,Sunny,0.0,60,6,1018
2,R-af1d28da,2019-02-03 06:00:00,63,7,Clear,0.0,72,6,1014
3,R-86eb239c,2019-01-30 12:00:00,59,6,Light drizzle,0.0,81,3,1015
4,R-7a0a08ef,2019-02-08 18:00:00,45,2,Sunny,0.0,76,6,1019
...,...,...,...,...,...,...,...,...,...
396144,R-b2334f87,2019-02-10 00:00:00,7,14,Sunny,0.0,51,6,1028
396145,R-ad9044ec,2019-01-29 00:00:00,73,8,Sunny,0.0,62,6,1019
396146,R-47c6c141,2019-01-01 06:00:00,63,4,Light rain shower,0.0,76,6,1007
396147,R-6b656856,2019-02-09 00:00:00,30,14,Sunny,0.0,64,6,1030


In [64]:
df_routes

,route_id,origin_id,destination_id,distance,average_hours
0,R-99cb7bba,C-01660979,C-b5ff31cd,156.500000,3.130000
1,R-ddac5f58,C-03bb3e48,C-56e39a5e,1994.609985,39.889999
2,R-f5b60fcb,C-927ceb5e,C-19236709,155.210007,3.100000
3,R-e55952e3,C-4fe0fa24,C-f5ed4c15,1351.000000,27.020000
4,R-b103319f,C-7212cebe,C-451776b7,1575.170044,31.500000
...,...,...,...,...,...
1805,R-974c21e5,C-2c9e75ef,C-d9e9d934,1205.439941,24.110001
1806,R-58b26646,C-6ea51d66,C-825b2625,887.590027,17.750000
1807,R-9ac999a7,C-c4565ee8,C-5d86b887,336.059998,6.720000
1808,R-ec3ad78d,C-d3bb431c,C-4fe0fa24,1103.329956,22.070000


In [65]:
df_truck_schedule1

,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 06:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 12:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-01-31 18:00:00
0,28411080,R-ef177918,2019-01-31 06:00:00,2019-02-01 00:00:00,0,2019-02-01 00:00:00
1,13193058,R-19a5b1a5,2019-01-26 06:00:00,2019-01-27 12:00:00,1,2019-01-26 06:00:00
...,...,...,...,...,...,...
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-05 18:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 00:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 06:00:00
12307,25704375,R-a2de5a9e,2019-02-05 06:00:00,2019-02-06 18:00:00,1,2019-02-06 12:00:00


In [66]:
nearest_hour_schedule_df = df_truck_schedule.copy()

# Step 2: Round 'estimated_arrival' and 'departure_date' to the nearest hour
nearest_hour_schedule_df['estimated_arrival_nearest_hour'] = nearest_hour_schedule_df['estimated_arrival'].dt.round(
    "H")
nearest_hour_schedule_df['departure_date_nearest_hour'] = nearest_hour_schedule_df['departure_date'].dt.round(
    "H")

# Step 3: Merge with routes DataFrame to get origin and destination
nearest_hour_schedule_route_df = pd.merge(
    nearest_hour_schedule_df,
    df_routes,
    on='route_id',
    how='left'
)

# Display the resulting DataFrame
nearest_hour_schedule_route_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000
...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,2019-01-07 05:00:00,2019-01-01 07:00:00,NaN,NaN,NaN,NaN
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000


In [67]:
nearest_hour_schedule_route_df.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                         859
destination_id                    859
distance                          859
average_hours                     859
dtype: int64

In [68]:
nearest_hour_schedule_route_df = nearest_hour_schedule_route_df.dropna()
nearest_hour_schedule_route_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000
...,...,...,...,...,...,...,...,...,...,...,...
12302,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000


In [69]:
nearest_hour_schedule_df.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
dtype: int64

In [70]:
origin_weather = df_city_weather.copy()
destination_weather = df_city_weather.copy()

In [71]:
# origin_weather = origin_weather.drop(columns=['datetime'])

In [72]:
# destination_weather = destination_weather.drop(columns=['datetime'])

In [73]:
origin_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-328bd8d3,14,16,Heavy snow,0.0,82,1,1019,2019-01-25 01:00:00
1,C-841ebdcb,54,6,Sunny,0.0,44,6,1023,2019-01-24 17:00:00
2,C-4fe0fa24,32,2,Patchy heavy snow,0.0,80,2,1015,2019-01-09 02:00:00
3,C-03bb3e48,-4,12,Partly cloudy,0.0,89,6,1023,2019-02-14 21:00:00
4,C-6df8beaf,36,13,Light sleet,0.0,95,6,1001,2019-01-28 05:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-90e30162,66,6,Light rain shower,0.0,95,6,1017,2019-02-05 21:00:00
51830,C-01660979,34,5,Light snow,0.0,47,6,1029,2019-02-12 00:00:00
51831,C-19236709,27,2,Overcast,0.0,85,6,1022,2019-02-01 03:00:00
51832,C-b5282c3b,9,18,Blizzard,0.0,75,0,1011,2019-01-29 14:00:00


In [74]:
df_routes

,route_id,origin_id,destination_id,distance,average_hours
0,R-99cb7bba,C-01660979,C-b5ff31cd,156.500000,3.130000
1,R-ddac5f58,C-03bb3e48,C-56e39a5e,1994.609985,39.889999
2,R-f5b60fcb,C-927ceb5e,C-19236709,155.210007,3.100000
3,R-e55952e3,C-4fe0fa24,C-f5ed4c15,1351.000000,27.020000
4,R-b103319f,C-7212cebe,C-451776b7,1575.170044,31.500000
...,...,...,...,...,...
1805,R-974c21e5,C-2c9e75ef,C-d9e9d934,1205.439941,24.110001
1806,R-58b26646,C-6ea51d66,C-825b2625,887.590027,17.750000
1807,R-9ac999a7,C-c4565ee8,C-5d86b887,336.059998,6.720000
1808,R-ec3ad78d,C-d3bb431c,C-4fe0fa24,1103.329956,22.070000


In [75]:
# origin_weather = pd.merge(df_routes, df_city_weather, how='left', left_on='origin_id', right_on='city_id', suffixes=('', '_origin'))

# Merging for destination city weather
# weather_info = pd.merge(origin_weather, df_city_weather, how='left', left_on='destination_id', right_on='city_id', suffixes=('', '_destination'))

# Selecting relevant columns

In [76]:
nearest_hour_schedule_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00
...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,2019-01-07 05:00:00,2019-01-01 07:00:00
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00


In [77]:
origin_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-328bd8d3,14,16,Heavy snow,0.0,82,1,1019,2019-01-25 01:00:00
1,C-841ebdcb,54,6,Sunny,0.0,44,6,1023,2019-01-24 17:00:00
2,C-4fe0fa24,32,2,Patchy heavy snow,0.0,80,2,1015,2019-01-09 02:00:00
3,C-03bb3e48,-4,12,Partly cloudy,0.0,89,6,1023,2019-02-14 21:00:00
4,C-6df8beaf,36,13,Light sleet,0.0,95,6,1001,2019-01-28 05:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-90e30162,66,6,Light rain shower,0.0,95,6,1017,2019-02-05 21:00:00
51830,C-01660979,34,5,Light snow,0.0,47,6,1029,2019-02-12 00:00:00
51831,C-19236709,27,2,Overcast,0.0,85,6,1022,2019-02-01 03:00:00
51832,C-b5282c3b,9,18,Blizzard,0.0,75,0,1011,2019-01-29 14:00:00


In [78]:
origin_weather_merge = pd.merge(nearest_hour_schedule_route_df,
                                origin_weather, left_on=['origin_id', 'departure_date_nearest_hour'], right_on=['city_id', 'datetime'], how='left')

In [79]:
origin_weather_merge.shape

(11449, 20)

In [80]:
origin_weather_merge.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
city_id                           552
temp                              552
wind_speed                        552
description                       552
precip                            552
humidity                          552
visibility                        552
pressure                          552
datetime                          552
dtype: int64

In [81]:
origin_weather_merge = origin_weather_merge.dropna()

In [82]:
origin_weather_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
city_id                           0
temp                              0
wind_speed                        0
description                       0
precip                            0
humidity                          0
visibility                        0
pressure                          0
datetime                          0
dtype: int64

In [83]:
origin_weather_merge.shape

(10897, 20)

In [84]:
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000,C-e6dcda92,14.0,6.0,Patchy heavy snow,0.0,94.0,2.0,1021.0,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000,C-b6e04c88,30.0,4.0,Heavy snow,0.0,95.0,1.0,1025.0,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000,C-7212cebe,-8.0,2.0,Partly cloudy,0.0,90.0,6.0,1026.0,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000,C-b5282c3b,10.0,14.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-01-28 07:00:00
11445,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000,C-e5bfb4e5,25.0,12.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-02-12 07:00:00
11446,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001,C-2c9e75ef,39.0,11.0,Sunny,0.0,51.0,6.0,1028.0,2019-01-10 07:00:00
11447,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000,C-328bd8d3,37.0,12.0,Partly cloudy,0.0,95.0,6.0,1018.0,2019-02-03 07:00:00


In [85]:
origin_weather_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours', 'city_id',
       'temp', 'wind_speed', 'description', 'precip', 'humidity', 'visibility',
       'pressure', 'datetime'],
      dtype='object')

In [86]:
origin_weather_merge = origin_weather_merge.dropna()
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000,C-e6dcda92,14.0,6.0,Patchy heavy snow,0.0,94.0,2.0,1021.0,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000,C-b6e04c88,30.0,4.0,Heavy snow,0.0,95.0,1.0,1025.0,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000,C-7212cebe,-8.0,2.0,Partly cloudy,0.0,90.0,6.0,1026.0,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000,C-b5282c3b,10.0,14.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-01-28 07:00:00
11445,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000,C-e5bfb4e5,25.0,12.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-02-12 07:00:00
11446,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001,C-2c9e75ef,39.0,11.0,Sunny,0.0,51.0,6.0,1028.0,2019-01-10 07:00:00
11447,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000,C-328bd8d3,37.0,12.0,Partly cloudy,0.0,95.0,6.0,1018.0,2019-02-03 07:00:00


In [87]:
# origin_weather_merge = origin_weather_merge.drop(columns='date')
origin_weather_merge = origin_weather_merge.drop_duplicates()
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000,C-e6dcda92,14.0,6.0,Patchy heavy snow,0.0,94.0,2.0,1021.0,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000,C-b6e04c88,30.0,4.0,Heavy snow,0.0,95.0,1.0,1025.0,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000,C-7212cebe,-8.0,2.0,Partly cloudy,0.0,90.0,6.0,1026.0,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000,C-b5282c3b,10.0,14.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-01-28 07:00:00
11445,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000,C-e5bfb4e5,25.0,12.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-02-12 07:00:00
11446,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001,C-2c9e75ef,39.0,11.0,Sunny,0.0,51.0,6.0,1028.0,2019-01-10 07:00:00
11447,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000,C-328bd8d3,37.0,12.0,Partly cloudy,0.0,95.0,6.0,1018.0,2019-02-03 07:00:00


In [88]:
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000,C-e6dcda92,14.0,6.0,Patchy heavy snow,0.0,94.0,2.0,1021.0,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000,C-b6e04c88,30.0,4.0,Heavy snow,0.0,95.0,1.0,1025.0,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000,C-7212cebe,-8.0,2.0,Partly cloudy,0.0,90.0,6.0,1026.0,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000,C-b5282c3b,10.0,14.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-01-28 07:00:00
11445,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000,C-e5bfb4e5,25.0,12.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-02-12 07:00:00
11446,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001,C-2c9e75ef,39.0,11.0,Sunny,0.0,51.0,6.0,1028.0,2019-01-10 07:00:00
11447,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000,C-328bd8d3,37.0,12.0,Partly cloudy,0.0,95.0,6.0,1018.0,2019-02-03 07:00:00


In [89]:
origin_weather_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours', 'city_id',
       'temp', 'wind_speed', 'description', 'precip', 'humidity', 'visibility',
       'pressure', 'datetime'],
      dtype='object')

In [90]:
origin_weather_merge.rename(columns={
    'temp': 'origin_temp',
    'wind_speed': 'origin_wind_speed',
    'description': 'origin_description',
    'humidity': 'origin_humidity',
    'pressure': 'origin_pressure',
    'visibility': 'origin_visibility',
    'precip': 'origin_precip'
}, inplace=True)

In [91]:
# origin_weather_merge = origin_weather_merge.drop(columns='datetime')
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,origin_temp,origin_wind_speed,origin_description,origin_precip,origin_humidity,origin_visibility,origin_pressure,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000,C-e6dcda92,14.0,6.0,Patchy heavy snow,0.0,94.0,2.0,1021.0,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000,C-b6e04c88,30.0,4.0,Heavy snow,0.0,95.0,1.0,1025.0,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000,C-7212cebe,-8.0,2.0,Partly cloudy,0.0,90.0,6.0,1026.0,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000,C-b5282c3b,10.0,14.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-01-28 07:00:00
11445,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000,C-e5bfb4e5,25.0,12.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-02-12 07:00:00
11446,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001,C-2c9e75ef,39.0,11.0,Sunny,0.0,51.0,6.0,1028.0,2019-01-10 07:00:00
11447,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000,C-328bd8d3,37.0,12.0,Partly cloudy,0.0,95.0,6.0,1018.0,2019-02-03 07:00:00


In [92]:
origin_weather_merge.isnull().any()

truck_id                          False
route_id                          False
departure_date                    False
estimated_arrival                 False
delay                             False
estimated_arrival_nearest_hour    False
departure_date_nearest_hour       False
origin_id                         False
destination_id                    False
distance                          False
average_hours                     False
city_id                           False
origin_temp                       False
origin_wind_speed                 False
origin_description                False
origin_precip                     False
origin_humidity                   False
origin_visibility                 False
origin_pressure                   False
datetime                          False
dtype: bool

In [93]:
origin_weather_merge = origin_weather_merge.drop_duplicates()

In [94]:
origin_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,average_hours,city_id,origin_temp,origin_wind_speed,origin_description,origin_precip,origin_humidity,origin_visibility,origin_pressure,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,12.410000,C-280b55fb,-6.0,10.0,Partly cloudy,0.0,59.0,6.0,1032.0,2019-01-31 07:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,26.150000,C-e6dcda92,14.0,6.0,Patchy heavy snow,0.0,94.0,2.0,1021.0,2019-01-26 07:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,3.710000,C-b6e04c88,30.0,4.0,Heavy snow,0.0,95.0,1.0,1025.0,2019-01-13 07:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,8.290000,C-7212cebe,-8.0,2.0,Partly cloudy,0.0,90.0,6.0,1026.0,2019-01-25 07:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,4.370000,C-b5282c3b,34.0,11.0,Sunny,0.0,81.0,6.0,1010.0,2019-01-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11444,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,6.280000,C-b5282c3b,10.0,14.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-01-28 07:00:00
11445,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,8.400000,C-e5bfb4e5,25.0,12.0,Heavy snow,0.0,95.0,1.0,1010.0,2019-02-12 07:00:00
11446,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,16.360001,C-2c9e75ef,39.0,11.0,Sunny,0.0,51.0,6.0,1028.0,2019-01-10 07:00:00
11447,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,10.380000,C-328bd8d3,37.0,12.0,Partly cloudy,0.0,95.0,6.0,1018.0,2019-02-03 07:00:00


In [95]:
origin_weather_merge = origin_weather_merge.drop_duplicates()
origin_weather_merge.shape

(10897, 20)

In [96]:
destination_weather

,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime
0,C-328bd8d3,14,16,Heavy snow,0.0,82,1,1019,2019-01-25 01:00:00
1,C-841ebdcb,54,6,Sunny,0.0,44,6,1023,2019-01-24 17:00:00
2,C-4fe0fa24,32,2,Patchy heavy snow,0.0,80,2,1015,2019-01-09 02:00:00
3,C-03bb3e48,-4,12,Partly cloudy,0.0,89,6,1023,2019-02-14 21:00:00
4,C-6df8beaf,36,13,Light sleet,0.0,95,6,1001,2019-01-28 05:00:00
...,...,...,...,...,...,...,...,...,...
51829,C-90e30162,66,6,Light rain shower,0.0,95,6,1017,2019-02-05 21:00:00
51830,C-01660979,34,5,Light snow,0.0,47,6,1029,2019-02-12 00:00:00
51831,C-19236709,27,2,Overcast,0.0,85,6,1022,2019-02-01 03:00:00
51832,C-b5282c3b,9,18,Blizzard,0.0,75,0,1011,2019-01-29 14:00:00


In [97]:
final_merge = pd.merge(origin_weather_merge, destination_weather, left_on=['destination_id', 'estimated_arrival_nearest_hour'],
                       right_on=['city_id', 'datetime'], how='left', suffixes=('_origin', '_destination'))

In [98]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,datetime_origin,city_id_destination,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime_destination
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,2019-01-31 07:00:00,C-ff8c0c3c,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,2019-01-26 07:00:00,C-451776b7,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,2019-01-13 07:00:00,C-a9f2c329,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,2019-01-25 07:00:00,C-d80a1e7d,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,2019-01-04 07:00:00,C-d80a1e7d,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,2019-01-28 07:00:00,C-7212cebe,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00
10893,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,2019-02-12 07:00:00,C-280b55fb,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00
10894,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,2019-01-10 07:00:00,C-328bd8d3,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00
10895,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,2019-02-03 07:00:00,C-2bd47dc5,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00


In [99]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'city_id_origin', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'datetime_origin',
       'city_id_destination', 'temp', 'wind_speed', 'description', 'precip',
       'humidity', 'visibility', 'pressure', 'datetime_destination'],
      dtype='object')

In [100]:
final_merge = final_merge.dropna()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,datetime_origin,city_id_destination,temp,wind_speed,description,precip,humidity,visibility,pressure,datetime_destination
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,2019-01-31 07:00:00,C-ff8c0c3c,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,2019-01-26 07:00:00,C-451776b7,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,2019-01-13 07:00:00,C-a9f2c329,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,2019-01-25 07:00:00,C-d80a1e7d,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,2019-01-04 07:00:00,C-d80a1e7d,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,2019-01-28 07:00:00,C-7212cebe,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00
10893,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,2019-02-12 07:00:00,C-280b55fb,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00
10894,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,2019-01-10 07:00:00,C-328bd8d3,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00
10895,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,2019-02-03 07:00:00,C-2bd47dc5,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00


In [101]:
final_merge.rename(columns={
    'temp': 'dest_temp',
    'wind_speed': 'dest_wind_speed',
    'description': 'dest_description',
    'humidity': 'dest_humidity',
    'pressure': 'dest_pressure',
    'precip': 'dest_precip',
    'visibility': 'dest_visibility'
}, inplace=True)

2024-10-06 13:28:12,587 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [102]:
final_merge = final_merge.drop(
    columns=['city_id_origin', 'city_id_destination'])
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,datetime_origin,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,1032.0,2019-01-31 07:00:00,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,1021.0,2019-01-26 07:00:00,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,1025.0,2019-01-13 07:00:00,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,1026.0,2019-01-25 07:00:00,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,1010.0,2019-01-04 07:00:00,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,1010.0,2019-01-28 07:00:00,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00
10893,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,1010.0,2019-02-12 07:00:00,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00
10894,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,1028.0,2019-01-10 07:00:00,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00
10895,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,1018.0,2019-02-03 07:00:00,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00


In [103]:
final_merge = final_merge.drop_duplicates()
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,datetime_origin,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,1032.0,2019-01-31 07:00:00,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,1021.0,2019-01-26 07:00:00,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,1025.0,2019-01-13 07:00:00,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,1026.0,2019-01-25 07:00:00,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,1010.0,2019-01-04 07:00:00,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,1010.0,2019-01-28 07:00:00,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00
10893,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,1010.0,2019-02-12 07:00:00,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00
10894,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,1028.0,2019-01-10 07:00:00,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00
10895,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,1018.0,2019-02-03 07:00:00,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00


In [104]:
final_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'origin_temp', 'origin_wind_speed', 'origin_description',
       'origin_precip', 'origin_humidity', 'origin_visibility',
       'origin_pressure', 'datetime_origin', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'datetime_destination'],
      dtype='object')

In [105]:
final_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
datetime_origin                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
datetime_destination              0
dtype: int64

In [106]:
schedule_data = df_truck_schedule.copy()

In [107]:
schedule_data

,truck_id,route_id,departure_date,estimated_arrival,delay
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0
...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0


In [108]:
schedule_data['departure_date'] = pd.to_datetime(
    schedule_data['departure_date'])
schedule_data['estimated_arrival'] = pd.to_datetime(
    schedule_data['estimated_arrival'])

# Round to nearest hour
schedule_data['departure_date'] = schedule_data['departure_date'].dt.round('H')
schedule_data['estimated_arrival'] = schedule_data['estimated_arrival'].dt.round(
    'H')

# Display the updated DataFrame
schedule_data

,truck_id,route_id,departure_date,estimated_arrival,delay
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:00:00,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:00:00,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 11:00:00,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:00:00,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:00:00,0
...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:00:00,1
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:00:00,0
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:00:00,1
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:00:00,0


In [109]:
hourly_exploded_scheduled_df = (nearest_hour_schedule_df.assign(custom_date=[pd.date_range(start, end, freq='H')  # Create custom date ranges
                                                                             for start, end
                                                                             # Using departure and estimated arrival times
                                                                             in zip(nearest_hour_schedule_df['departure_date'], nearest_hour_schedule_df['estimated_arrival'])])
                                # Explode the DataFrame based on the custom date range
                                .explode('custom_date', ignore_index=True))

In [110]:
df_traffic['datetime'] = pd.to_datetime(df_traffic['datetime'])

In [111]:
df_traffic

,route_id,no_of_vehicles,accident,datetime
0,R-2278e851,522.0,0,2019-01-03 04:00:00
1,R-0e3ff935,216.0,0,2019-02-09 00:00:00
2,R-4c68ff74,425.0,1,2019-01-18 05:00:00
3,R-2d8055b0,2602.0,0,2019-01-21 16:00:00
4,R-a08baf8e,565.0,0,2019-01-31 02:00:00
...,...,...,...,...
2597908,R-3cad4a95,684.0,0,2019-01-14 06:00:00
2597909,R-ad451391,828.0,1,2019-01-18 03:00:00
2597910,R-0685acb5,1900.0,0,2019-01-09 11:00:00
2597911,R-fd52f991,780.0,0,2019-02-07 05:00:00


In [112]:
# df_traffic['end_time'] = df_traffic['datetime'] + pd.Timedelta(hours=24)

In [113]:
# df_traffic = (df_traffic.assign(custom_date=[
#     pd.date_range(start, end, freq='6H')  # Create hourly date ranges
#     for start, end in zip(df_traffic['datetime'], df_traffic['end_time'])
# ]).explode('custom_date', ignore_index=True))

In [114]:
hourly_exploded_scheduled_df.head()

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 07:00:00
1,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 08:00:00
2,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 09:00:00
3,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 10:00:00
4,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 11:00:00


In [115]:
hourly_exploded_scheduled_df = hourly_exploded_scheduled_df.drop_duplicates()

In [116]:
hourly_exploded_scheduled_df

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 07:00:00
1,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 08:00:00
2,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 09:00:00
3,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 10:00:00
4,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 11:00:00
...,...,...,...,...,...,...,...,...
311568,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 08:00:00
311569,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 09:00:00
311570,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 10:00:00
311571,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 11:00:00


In [117]:
df_traffic

,route_id,no_of_vehicles,accident,datetime
0,R-2278e851,522.0,0,2019-01-03 04:00:00
1,R-0e3ff935,216.0,0,2019-02-09 00:00:00
2,R-4c68ff74,425.0,1,2019-01-18 05:00:00
3,R-2d8055b0,2602.0,0,2019-01-21 16:00:00
4,R-a08baf8e,565.0,0,2019-01-31 02:00:00
...,...,...,...,...
2597908,R-3cad4a95,684.0,0,2019-01-14 06:00:00
2597909,R-ad451391,828.0,1,2019-01-18 03:00:00
2597910,R-0685acb5,1900.0,0,2019-01-09 11:00:00
2597911,R-fd52f991,780.0,0,2019-02-07 05:00:00


In [118]:
scheduled_traffic = hourly_exploded_scheduled_df.merge(df_traffic, left_on=[
                                                       'route_id', 'custom_date'], right_on=['route_id', 'datetime'], how='left')

In [119]:
scheduled_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,no_of_vehicles,accident,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 07:00:00,2116.0,0,2019-01-31 07:00:00
1,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 08:00:00,2184.0,0,2019-01-31 08:00:00
2,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 09:00:00,2590.0,0,2019-01-31 09:00:00
3,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 10:00:00,1921.0,0,2019-01-31 10:00:00
4,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 11:00:00,2346.0,0,2019-01-31 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
311673,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 08:00:00,2392.0,0,2019-02-06 08:00:00
311674,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 09:00:00,2581.0,0,2019-02-06 09:00:00
311675,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 10:00:00,2529.0,0,2019-02-06 10:00:00
311676,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 11:00:00,2284.0,0,2019-02-06 11:00:00


In [120]:
scheduled_traffic.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
custom_date                       0
no_of_vehicles                    0
accident                          0
datetime                          0
dtype: int64

In [121]:
scheduled_traffic = scheduled_traffic.dropna()
scheduled_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,no_of_vehicles,accident,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 07:00:00,2116.0,0,2019-01-31 07:00:00
1,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 08:00:00,2184.0,0,2019-01-31 08:00:00
2,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 09:00:00,2590.0,0,2019-01-31 09:00:00
3,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 10:00:00,1921.0,0,2019-01-31 10:00:00
4,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 11:00:00,2346.0,0,2019-01-31 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
311673,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 08:00:00,2392.0,0,2019-02-06 08:00:00
311674,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 09:00:00,2581.0,0,2019-02-06 09:00:00
311675,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 10:00:00,2529.0,0,2019-02-06 10:00:00
311676,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 11:00:00,2284.0,0,2019-02-06 11:00:00


In [122]:
scheduled_traffic = scheduled_traffic.drop_duplicates()
scheduled_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,no_of_vehicles,accident,datetime
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 07:00:00,2116.0,0,2019-01-31 07:00:00
1,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 08:00:00,2184.0,0,2019-01-31 08:00:00
2,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 09:00:00,2590.0,0,2019-01-31 09:00:00
3,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 10:00:00,1921.0,0,2019-01-31 10:00:00
4,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,2019-01-31 11:00:00,2346.0,0,2019-01-31 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...
311673,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 08:00:00,2392.0,0,2019-02-06 08:00:00
311674,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 09:00:00,2581.0,0,2019-02-06 09:00:00
311675,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 10:00:00,2529.0,0,2019-02-06 10:00:00
311676,25704375,R-a2de5a9e,2019-02-05 07:00:00,2019-02-06 12:45:36,1,2019-02-06 13:00:00,2019-02-05 07:00:00,2019-02-06 11:00:00,2284.0,0,2019-02-06 11:00:00


In [123]:
scheduled_traffic.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
custom_date                       0
no_of_vehicles                    0
accident                          0
datetime                          0
dtype: int64

In [124]:
def custom_agg(values):
    if any(values == 1):
        return 1
    else:
        return 0

In [125]:
scheduled_route_traffic = scheduled_traffic.groupby(['truck_id', 'route_id'], as_index=False).agg(
    avg_no_of_vehicles=('no_of_vehicles', 'mean'),
    accident=('accident', custom_agg)
)

In [126]:
scheduled_route_traffic

,truck_id,route_id,avg_no_of_vehicles,accident
0,10008392,R-06dfe39e,1877.916626,1
1,10008392,R-35dca729,1820.098022,1
2,10008392,R-51d1a32a,1536.542358,1
3,10008392,R-7ab5ae85,1927.824951,1
4,10008392,R-c6f8e036,829.283325,1
...,...,...,...,...
12235,99981667,R-abbf83d1,1792.812500,0
12236,99981667,R-e9bf94ff,2074.166748,0
12237,99981667,R-ed1094fd,1976.812500,1
12238,99981667,R-f363ad37,1831.750000,0


In [127]:
scheduled_route_traffic.isnull().sum()

truck_id              0
route_id              0
avg_no_of_vehicles    0
accident              0
dtype: int64

In [128]:
final_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,origin_pressure,datetime_origin,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,1032.0,2019-01-31 07:00:00,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,1021.0,2019-01-26 07:00:00,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,1025.0,2019-01-13 07:00:00,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,1026.0,2019-01-25 07:00:00,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,1010.0,2019-01-04 07:00:00,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10892,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,1010.0,2019-01-28 07:00:00,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00
10893,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,1010.0,2019-02-12 07:00:00,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00
10894,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,1028.0,2019-01-10 07:00:00,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00
10895,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,1018.0,2019-02-03 07:00:00,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00


In [129]:
final_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
datetime_origin                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
datetime_destination              0
dtype: int64

In [130]:
scheduled_route_traffic

,truck_id,route_id,avg_no_of_vehicles,accident
0,10008392,R-06dfe39e,1877.916626,1
1,10008392,R-35dca729,1820.098022,1
2,10008392,R-51d1a32a,1536.542358,1
3,10008392,R-7ab5ae85,1927.824951,1
4,10008392,R-c6f8e036,829.283325,1
...,...,...,...,...
12235,99981667,R-abbf83d1,1792.812500,0
12236,99981667,R-e9bf94ff,2074.166748,0
12237,99981667,R-ed1094fd,1976.812500,1
12238,99981667,R-f363ad37,1831.750000,0


In [131]:
origin_destination_weather_traffic_merge = final_merge.merge(
    scheduled_route_traffic, on=['truck_id', 'route_id'], how='left')

In [132]:
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00,179.000000,1
10702,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0
10703,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0
10704,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1


In [133]:
origin_destination_weather_traffic_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
datetime_origin                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
datetime_destination              0
avg_no_of_vehicles          

In [134]:
origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.dropna()

In [135]:
# origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.drop(columns='index_y')

In [136]:
origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.drop_duplicates()
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00,179.000000,1
10702,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0
10703,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0
10704,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1


In [137]:
origin_destination_weather_traffic_merge.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'estimated_arrival_nearest_hour', 'departure_date_nearest_hour',
       'origin_id', 'destination_id', 'distance', 'average_hours',
       'origin_temp', 'origin_wind_speed', 'origin_description',
       'origin_precip', 'origin_humidity', 'origin_visibility',
       'origin_pressure', 'datetime_origin', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'datetime_destination', 'avg_no_of_vehicles',
       'accident'],
      dtype='object')

In [138]:
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00,179.000000,1
10702,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0
10703,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0
10704,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1


In [139]:
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,6.000000,Clear
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,3.000000,Fog
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,6.000000,Cloudy
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,6.000000,Sunny
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,4.500000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,4.000000,Heavy rain
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,5.440000,Clear
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,6.000000,Clear
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,4.333333,Clear


In [140]:
schedule_weather_merge.isnull().sum()

truck_id                0
route_id                0
departure_date          0
estimated_arrival       0
delay                   0
route_avg_temp          0
route_avg_wind_speed    0
route_avg_humidity      0
route_avg_pressure      0
route_avg_precip        0
route_avg_visibility    0
route_description       0
dtype: int64

In [141]:
schedule_weather_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,route_avg_visibility,route_description
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,6.000000,Clear
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,3.000000,Fog
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,6.000000,Cloudy
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,6.000000,Sunny
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,4.500000,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...
12303,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,4.000000,Heavy rain
12304,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,5.440000,Clear
12305,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,6.000000,Clear
12306,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,4.333333,Clear


In [142]:
merged_data_weather_traffic = pd.merge(schedule_weather_merge, origin_destination_weather_traffic_merge, on=[
                                       'truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], how='left')

In [143]:
origin_destination_weather_traffic_merge

,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,origin_id,destination_id,distance,...,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,2019-01-31 19:00:00,2019-01-31 07:00:00,C-280b55fb,C-ff8c0c3c,620.469971,...,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,2019-01-27 09:00:00,2019-01-26 07:00:00,C-e6dcda92,C-451776b7,1307.650024,...,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,2019-01-13 11:00:00,2019-01-13 07:00:00,C-b6e04c88,C-a9f2c329,185.729996,...,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,2019-01-25 15:00:00,2019-01-25 07:00:00,C-7212cebe,C-d80a1e7d,414.269989,...,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,2019-01-04 11:00:00,2019-01-04 07:00:00,C-b5282c3b,C-d80a1e7d,218.309998,...,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10701,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,2019-01-28 13:00:00,2019-01-28 07:00:00,C-b5282c3b,C-7212cebe,314.049988,...,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00,179.000000,1
10702,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,2019-02-12 15:00:00,2019-02-12 07:00:00,C-e5bfb4e5,C-280b55fb,420.079987,...,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0
10703,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,2019-01-10 23:00:00,2019-01-10 07:00:00,C-2c9e75ef,C-328bd8d3,817.859985,...,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0
10704,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,2019-02-03 17:00:00,2019-02-03 07:00:00,C-328bd8d3,C-2bd47dc5,518.940002,...,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1


In [144]:
origin_destination_weather_traffic_merge.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
datetime_origin                   0
dest_temp                         0
dest_wind_speed                   0
dest_description                  0
dest_precip                       0
dest_humidity                     0
dest_visibility                   0
dest_pressure                     0
datetime_destination              0
avg_no_of_vehicles          

In [145]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0.0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1.0
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0.0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1.0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12260,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0.0
12261,74796611,R-c084fe21,2019-01-01 07:00:00,2019-01-07 05:23:24,0,42.640000,7.800000,78.480000,1015.680000,0.020000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
12262,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0.0
12263,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1.0


In [146]:
merged_data_weather_traffic.isnull().sum()

truck_id                             0
route_id                             0
departure_date                       0
estimated_arrival                    0
delay                                0
route_avg_temp                       0
route_avg_wind_speed                 0
route_avg_humidity                   0
route_avg_pressure                   0
route_avg_precip                     0
route_avg_visibility                 0
route_description                    0
estimated_arrival_nearest_hour    1600
departure_date_nearest_hour       1600
origin_id                         1600
destination_id                    1600
distance                          1600
average_hours                     1600
origin_temp                       1600
origin_wind_speed                 1600
origin_description                1600
origin_precip                     1600
origin_humidity                   1600
origin_visibility                 1600
origin_pressure                   1600
datetime_origin          

In [147]:
merged_data_weather_traffic = merged_data_weather_traffic.dropna()

In [148]:
merged_data_weather_traffic = merged_data_weather_traffic.drop_duplicates()

In [149]:
merged_data_weather_traffic

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,dest_temp,dest_wind_speed,dest_description,dest_precip,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,34.0,5.0,Partly cloudy,0.0,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0.0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,70.0,2.0,Sunny,0.0,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1.0
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,28.0,6.0,Cloudy,0.0,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0.0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,19.0,9.0,Partly cloudy,0.0,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1.0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,43.0,5.0,Sunny,0.0,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12259,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,10.0,9.0,Overcast,0.0,88.0,5.0,1014.0,2019-01-28 13:00:00,179.000000,1.0
12260,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,41.0,12.0,Light rain,0.0,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0.0
12262,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,23.0,4.0,Clear,0.0,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0.0
12263,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,39.0,4.0,Sunny,0.0,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1.0


In [150]:
merged_data_weather_traffic_trucks = pd.merge(
    merged_data_weather_traffic, df_trucks, on='truck_id', how='left')

In [151]:
df_trucks

,truck_id,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,29303248,9,4000.0,15,gas
1,29292035,9,6000.0,24,diesel
2,17287024,12,15000.0,17,gas
3,11098873,11,15000.0,20,gas
4,22171671,8,20000.0,25,diesel
...,...,...,...,...,...
1291,67906530,11,6000.0,22,diesel
1292,11072962,7,15000.0,20,diesel
1293,29780011,15,4000.0,19,gas
1294,32628685,11,15000.0,18,gas


In [152]:
merged_data_weather_traffic_trucks.isnull().sum()

truck_id                           0
route_id                           0
departure_date                     0
estimated_arrival                  0
delay                              0
route_avg_temp                     0
route_avg_wind_speed               0
route_avg_humidity                 0
route_avg_pressure                 0
route_avg_precip                   0
route_avg_visibility               0
route_description                  0
estimated_arrival_nearest_hour     0
departure_date_nearest_hour        0
origin_id                          0
destination_id                     0
distance                           0
average_hours                      0
origin_temp                        0
origin_wind_speed                  0
origin_description                 0
origin_precip                      0
origin_humidity                    0
origin_visibility                  0
origin_pressure                    0
datetime_origin                    0
dest_temp                          0
d

In [153]:
merged_data_weather_traffic_trucks = merged_data_weather_traffic_trucks.dropna()

In [154]:
merged_data_weather_traffic_trucks

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,dest_humidity,dest_visibility,dest_pressure,datetime_destination,avg_no_of_vehicles,accident,truck_age,load_capacity_pounds,mileage_mpg,fuel_type
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,62.0,6.0,1019.0,2019-01-31 19:00:00,2189.230713,0.0,9.0,3000.0,22.0,diesel
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,35.0,6.0,1021.0,2019-01-27 09:00:00,1599.036987,1.0,12.0,4000.0,22.0,diesel
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,89.0,6.0,1031.0,2019-01-13 11:00:00,144.500000,0.0,9.0,15000.0,27.0,diesel
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,32.0,6.0,1026.0,2019-01-25 15:00:00,2174.111084,1.0,9.0,3000.0,18.0,gas
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,53.0,6.0,1010.0,2019-01-04 11:00:00,2194.199951,0.0,8.0,15000.0,18.0,gas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10660,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,88.0,5.0,1014.0,2019-01-28 13:00:00,179.000000,1.0,9.0,6000.0,15.0,gas
10661,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,97.0,6.0,1000.0,2019-02-12 15:00:00,151.000000,0.0,10.0,3000.0,19.0,gas
10662,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,75.0,6.0,1029.0,2019-01-10 23:00:00,2150.882324,0.0,10.0,6000.0,15.0,gas
10663,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,84.0,6.0,1021.0,2019-02-03 17:00:00,2245.636475,1.0,10.0,3000.0,17.0,gas


In [155]:
df_drivers.isnull().sum()

driver_id            0
name                 0
gender               0
age                  0
experience           0
driving_style        0
ratings              0
vehicle_no           0
average_speed_mph    0
dtype: int64

In [156]:
final_merge_dataset = pd.merge(merged_data_weather_traffic_trucks,
                               df_drivers, left_on='truck_id', right_on='vehicle_no', how='left')

In [157]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,fuel_type,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,diesel,b950f2d6-0,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,diesel,99525ce4-b,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,diesel,c249bfc7-9,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,gas,aa8ea26b-e,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,gas,4d2e08bb-8,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,gas,c19bf3c7-f,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,gas,35e07ef6-b,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,gas,45e3916e-1,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,gas,d9f30553-6,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001


In [158]:
def has_midnight(start, end):
    return int(start.date() != end.date())

In [159]:
final_merge_dataset['is_midnight'] = final_merge_dataset.apply(
    lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [160]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,b950f2d6-0,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,99525ce4-b,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,c249bfc7-9,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,aa8ea26b-e,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,4d2e08bb-8,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,c19bf3c7-f,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,35e07ef6-b,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,45e3916e-1,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,d9f30553-6,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0


In [161]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'datetime_origin', 'dest_temp',
       'dest_wind_speed', 'dest_description', 'dest_precip', 'dest_humidity',
       'dest_visibility', 'dest_pressure', 'datetime_destination',
       'avg_no_of_vehicles', 'accident', 'truck_age', 'load_capacity_pounds',
       'mileage_mpg', 'fuel_type', 'driver_id', 'name', 'gender', 'age',
       'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight'],
      dtype='object')

In [162]:
final_merge_dataset.shape

(10611, 50)

In [163]:
final_merge_dataset.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
route_avg_temp                      0
route_avg_wind_speed                0
route_avg_humidity                  0
route_avg_pressure                  0
route_avg_precip                    0
route_avg_visibility                0
route_description                   0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
origin_temp                         0
origin_wind_speed                   0
origin_description                  0
origin_precip                       0
origin_humidity                     0
origin_visibility                   0
origin_pressure                     0
datetime_origin                     0
dest_temp   

In [164]:
final_merge_dataset = final_merge_dataset.drop(
    columns=['datetime_destination', 'datetime_origin'])
final_merge_dataset.isnull().sum()

truck_id                            0
route_id                            0
departure_date                      0
estimated_arrival                   0
delay                               0
route_avg_temp                      0
route_avg_wind_speed                0
route_avg_humidity                  0
route_avg_pressure                  0
route_avg_precip                    0
route_avg_visibility                0
route_description                   0
estimated_arrival_nearest_hour      0
departure_date_nearest_hour         0
origin_id                           0
destination_id                      0
distance                            0
average_hours                       0
origin_temp                         0
origin_wind_speed                   0
origin_description                  0
origin_precip                       0
origin_humidity                     0
origin_visibility                   0
origin_pressure                     0
dest_temp                           0
dest_wind_sp

In [165]:
final_merge_dataset = final_merge_dataset.dropna()
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,b950f2d6-0,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,99525ce4-b,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,c249bfc7-9,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,aa8ea26b-e,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,4d2e08bb-8,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,c19bf3c7-f,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,35e07ef6-b,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,45e3916e-1,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,d9f30553-6,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0


In [166]:
final_merge_dataset = final_merge_dataset.drop_duplicates()
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,b950f2d6-0,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,99525ce4-b,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,c249bfc7-9,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,aa8ea26b-e,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,4d2e08bb-8,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,c19bf3c7-f,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,35e07ef6-b,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,45e3916e-1,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,d9f30553-6,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0


In [167]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight'],
      dtype='object')

In [168]:
final_merge_dataset.shape

(10482, 48)

In [169]:
final_merge_dataset.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_humidity                0
route_avg_pressure                0
route_avg_precip                  0
route_avg_visibility              0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description            

In [170]:
final_merge_dataset = final_merge_dataset.dropna()
final_merge_dataset = final_merge_dataset.drop_duplicates()
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,b950f2d6-0,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,99525ce4-b,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,c249bfc7-9,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,aa8ea26b-e,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,4d2e08bb-8,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,c19bf3c7-f,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,35e07ef6-b,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,45e3916e-1,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,d9f30553-6,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0


In [171]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight'],
      dtype='object')

In [173]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,b950f2d6-0,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,99525ce4-b,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,c249bfc7-9,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,aa8ea26b-e,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,4d2e08bb-8,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,c19bf3c7-f,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,35e07ef6-b,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,45e3916e-1,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,d9f30553-6,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0


In [174]:
final_merge_dataset['unique_id'] = final_merge_dataset.index
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0,2
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0,3
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0,10606
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0,10607
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0,10608
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0,10609


In [175]:
# Check the shape of the DataFrame
print(final_merge_dataset.shape)  # This will show (10482, number_of_columns)


(10482, 49)


In [176]:
# Check if the indices are consecutive
print(final_merge_dataset.index)  # This will display the index values of the DataFrame


Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       10601, 10602, 10603, 10604, 10605, 10606, 10607, 10608, 10609, 10610],
      dtype='int64', length=10482)


In [177]:
final_merge_dataset = final_merge_dataset.reset_index(drop=True)
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0,2
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0,3
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10477,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0,10606
10478,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0,10607
10479,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0,10608
10480,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0,10609


In [178]:
# Find missing unique_ids
missing_ids = set(range(final_merge_dataset['unique_id'].max() + 1)) - set(final_merge_dataset['unique_id'])
print("Missing unique_ids:", missing_ids)


Missing unique_ids: {7169, 7681, 7684, 2055, 1547, 7693, 5662, 544, 7714, 36, 6180, 2599, 3113, 2094, 8751, 2096, 7729, 57, 2617, 5692, 9276, 580, 7238, 8263, 7241, 1103, 3663, 9809, 5203, 2133, 4709, 618, 7282, 4219, 3196, 4734, 7295, 128, 7299, 2693, 6278, 1672, 1163, 6800, 5265, 9360, 4762, 4763, 5790, 1696, 3236, 5286, 5799, 1192, 6827, 685, 1199, 4785, 6326, 3263, 716, 7372, 3790, 4303, 4309, 726, 9435, 4316, 225, 5857, 10471, 5356, 8943, 1784, 7416, 4350, 4865, 773, 7949, 9490, 5916, 4387, 4393, 10032, 3387, 4416, 4419, 1352, 5454, 1363, 9557, 3932, 3934, 7518, 3937, 5473, 7021, 1905, 9074, 1907, 376, 6536, 1961, 7599, 8111, 2484, 7612, 10176, 961, 4033, 5574, 8138, 7115, 5585, 2003, 8147, 7637, 8151, 2008, 3040, 3555, 6116, 3557, 7144, 7660, 7154, 1012, 507, 2044}


In [179]:
# Reset unique_id to remove gaps
final_merge_dataset['unique_id'] = range(len(final_merge_dataset))
final_merge_dataset


,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,Katie Armstrong,female,45.0,14.0,conservative,6.0,28411080.0,42.939999,0,0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,James Henderson,male,47.0,7.0,conservative,3.0,13193058.0,47.009998,1,1
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,Emily Foster,male,48.0,5.0,proactive,3.0,29077811.0,59.830002,0,2
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,Alexander Hoover,male,49.0,0.0,proactive,7.0,12492275.0,59.490002,0,3
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,Jeffrey Lambert,male,45.0,14.0,conservative,6.0,10784487.0,38.980000,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10477,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,Robert Mcdonald,male,52.0,13.0,conservative,8.0,59301367.0,46.560001,0,10477
10478,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,Jason Lopez,male,47.0,9.0,proactive,8.0,58330582.0,65.419998,0,10478
10479,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,Raymond Johnson,male,50.0,10.0,proactive,3.0,25670063.0,61.650002,0,10479
10480,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,Daniel Marks,male,47.0,5.0,proactive,7.0,42302347.0,62.220001,0,10480


In [171]:
final_merge_dataset.to_csv(
    '/Users/rammaruthi/Desktop/Truck/notebooks/final_merge_dataset.csv')

In [172]:
final_merge_dataset.isnull().sum()

truck_id                          0
route_id                          0
departure_date                    0
estimated_arrival                 0
delay                             0
route_avg_temp                    0
route_avg_wind_speed              0
route_avg_humidity                0
route_avg_pressure                0
route_avg_precip                  0
route_avg_visibility              0
route_description                 0
estimated_arrival_nearest_hour    0
departure_date_nearest_hour       0
origin_id                         0
destination_id                    0
distance                          0
average_hours                     0
origin_temp                       0
origin_wind_speed                 0
origin_description                0
origin_precip                     0
origin_humidity                   0
origin_visibility                 0
origin_pressure                   0
dest_temp                         0
dest_wind_speed                   0
dest_description            

In [173]:
# Checks for duplicates in the 'unique_id' column
final_merge_dataset['unique_id'].is_unique
final_merge_dataset['unique_id'].isnull().sum()  # Checks for missing IDs

0

In [174]:
final_merge_dataset['unique_id'].is_unique

True

In [175]:
missing_ids = set(range(1, 10608)) - set(final_merge_dataset['unique_id'])
missing_ids

set()

In [176]:
final_merge_dataset.shape

(10611, 49)

In [177]:
cts_cols = ['route_avg_temp', 'route_avg_wind_speed',
            'route_avg_precip', 'route_avg_humidity', 'route_avg_visibility',
            'route_avg_pressure', 'distance', 'average_hours',
            'origin_temp', 'origin_wind_speed', 'origin_precip', 'origin_humidity',
            'origin_visibility', 'origin_pressure',
            'dest_temp', 'dest_wind_speed', 'dest_precip',
            'dest_humidity', 'dest_visibility', 'dest_pressure',
            'avg_no_of_vehicles', 'truck_age', 'load_capacity_pounds', 'mileage_mpg',
            'age', 'experience', 'average_speed_mph']

In [178]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight', 'unique_id'],
      dtype='object')

In [179]:
cat_cols = ['route_description',
            'origin_description', 'dest_description',
            'accident', 'fuel_type',
            'gender', 'driving_style', 'ratings', 'is_midnight']

In [180]:
from sklearn.preprocessing import OneHotEncoder

In [181]:
pip install joblib


2024-10-04 16:03:13,361 WARNING: DeprecationWarning: This process (pid=12752) is multi-threaded, use of forkpty() may lead to deadlocks in the child.



Note: you may need to restart the kernel to use updated packages.


In [182]:
output_dir = '/Users/rammaruthi/Desktop/Truck/encoders'

In [183]:
import os
import sys
import joblib

In [184]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,Katie Armstrong,female,45,14.0,conservative,6,28411080,42.94,0,1
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,James Henderson,male,47,7.0,conservative,3,13193058,47.01,1,2
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,Emily Foster,male,48,5.0,proactive,3,29077811,59.83,0,3
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,Alexander Hoover,male,49,0.0,proactive,7,12492275,59.49,0,4
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,Jeffrey Lambert,male,45,14.0,conservative,6,10784487,38.98,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,Robert Mcdonald,male,52,13.0,conservative,8,59301367,46.56,0,10607
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,Jason Lopez,male,47,9.0,proactive,8,58330582,65.42,0,10608
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,Raymond Johnson,male,50,10.0,proactive,3,25670063,61.65,0,10609
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,Daniel Marks,male,47,5.0,proactive,7,42302347,62.22,0,10610


In [185]:
final_merge_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight,unique_id
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,Katie Armstrong,female,45,14.0,conservative,6,28411080,42.94,0,1
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,James Henderson,male,47,7.0,conservative,3,13193058,47.01,1,2
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,Emily Foster,male,48,5.0,proactive,3,29077811,59.83,0,3
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,Alexander Hoover,male,49,0.0,proactive,7,12492275,59.49,0,4
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,Jeffrey Lambert,male,45,14.0,conservative,6,10784487,38.98,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,Robert Mcdonald,male,52,13.0,conservative,8,59301367,46.56,0,10607
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,Jason Lopez,male,47,9.0,proactive,8,58330582,65.42,0,10608
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,Raymond Johnson,male,50,10.0,proactive,3,25670063,61.65,0,10609
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,Daniel Marks,male,47,5.0,proactive,7,42302347,62.22,0,10610


In [186]:
final_merge_dataset.columns

Index(['truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay',
       'route_avg_temp', 'route_avg_wind_speed', 'route_avg_humidity',
       'route_avg_pressure', 'route_avg_precip', 'route_avg_visibility',
       'route_description', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'origin_temp', 'origin_wind_speed',
       'origin_description', 'origin_precip', 'origin_humidity',
       'origin_visibility', 'origin_pressure', 'dest_temp', 'dest_wind_speed',
       'dest_description', 'dest_precip', 'dest_humidity', 'dest_visibility',
       'dest_pressure', 'avg_no_of_vehicles', 'accident', 'truck_age',
       'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'driver_id', 'name',
       'gender', 'age', 'experience', 'driving_style', 'ratings', 'vehicle_no',
       'average_speed_mph', 'is_midnight', 'unique_id'],
      dtype='object')

In [187]:
import os
import joblib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


def create_one_hot_encoders(dataframe, cat_cols, output_dir='/Users/rammaruthi/Desktop/Truck/encoders'):
    """
    Creates one-hot encoders for specified categorical columns in the DataFrame and returns the modified DataFrame.

    Parameters:
    - dataframe: pd.DataFrame, the input DataFrame containing categorical columns.
    - cat_cols: list of str, the categorical column names to encode.
    - output_dir: str, the directory where encoder files will be saved.

    Returns:
    - pd.DataFrame: the modified DataFrame with one-hot encoded columns.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create a copy of the DataFrame to avoid modifying the original one
    df_encoded = dataframe.copy()

    for col in cat_cols:
        # Initialize OneHotEncoder
        encoder = OneHotEncoder(drop='first', sparse_output=False)

        # Fit the encoder to the column and transform the data
        encoded_col = encoder.fit_transform(df_encoded[[col]])

        # Get the column names for the encoded features
        encoded_col_names = encoder.get_feature_names_out([col])

        # Create a DataFrame with the encoded columns
        encoded_df = pd.DataFrame(
            encoded_col, columns=encoded_col_names, index=df_encoded.index)

        # Drop the original column from the DataFrame
        df_encoded.drop(columns=[col], inplace=True)

        # Assign the encoded columns directly to the DataFrame
        df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

        # Save the encoder as a .pkl file
        joblib.dump(encoder, os.path.join(output_dir, f'{col}_encoder.pkl'))
        print(f'Saved encoder for {col} to {output_dir}/{col}_encoder.pkl')

    return df_encoded

# Example usage:

In [188]:
df_encoded = create_one_hot_encoders(final_merge_dataset, cat_cols)

Saved encoder for route_description to /Users/rammaruthi/Desktop/Truck/encoders/route_description_encoder.pkl
Saved encoder for origin_description to /Users/rammaruthi/Desktop/Truck/encoders/origin_description_encoder.pkl
Saved encoder for dest_description to /Users/rammaruthi/Desktop/Truck/encoders/dest_description_encoder.pkl
Saved encoder for accident to /Users/rammaruthi/Desktop/Truck/encoders/accident_encoder.pkl
Saved encoder for fuel_type to /Users/rammaruthi/Desktop/Truck/encoders/fuel_type_encoder.pkl
Saved encoder for gender to /Users/rammaruthi/Desktop/Truck/encoders/gender_encoder.pkl
Saved encoder for driving_style to /Users/rammaruthi/Desktop/Truck/encoders/driving_style_encoder.pkl
Saved encoder for ratings to /Users/rammaruthi/Desktop/Truck/encoders/ratings_encoder.pkl
Saved encoder for is_midnight to /Users/rammaruthi/Desktop/Truck/encoders/is_midnight_encoder.pkl


In [189]:
df_encoded.shape

(10611, 164)

In [190]:
from sklearn.preprocessing import StandardScaler

In [191]:
df_encoded

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,gender_male,driving_style_proactive,ratings_3,ratings_4,ratings_5,ratings_6,ratings_7,ratings_8,ratings_9,is_midnight_1
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,47.250000,9.750000,74.500000,1021.750000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,55.833333,5.166667,64.500000,1011.333333,0.000000,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,46.500000,7.500000,56.000000,1018.000000,0.000000,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,53.000000,10.500000,46.000000,1007.500000,0.000000,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,40.000000,9.000000,74.000000,1020.000000,0.000000,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,72.333333,10.666667,61.333333,1013.666667,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,48.666667,3.333333,71.333333,1012.000000,0.133333,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,53.500000,3.500000,91.750000,1015.250000,0.000000,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,64.000000,4.000000,83.333333,1012.333333,0.000000,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [192]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
import os


def save_scalers_and_scale(df, cts_cols, scalers_directory):
    """
    Scales specified columns in a DataFrame using StandardScaler
    and saves each scaler as a .pkl file.

    Parameters:
    - df: pd.DataFrame, the DataFrame containing the data.
    - cts_cols: list of str, columns to scale.
    - scalers_directory: str, path to the directory where scalers will be saved.

    Returns:
    - pd.DataFrame: the DataFrame with scaled columns.
    """

    # Create the directory if it doesn't exist
    os.makedirs(scalers_directory, exist_ok=True)

    # Loop through each column, create a StandardScaler, fit, transform, and save it
    for col in cts_cols:
        scaler = StandardScaler()  # Initialize StandardScaler

        # Fit the scaler to the column data and transform it
        df[col] = scaler.fit_transform(df[[col]])

        # Save the scaler for the column
        scaler_path = os.path.join(scalers_directory, f'{col}_scaler.pkl')
        joblib.dump(scaler, scaler_path)

        print(f'Scaler for column "{col}" saved to {scaler_path}')

    return df  # Return the modified DataFrame

# Example usage:
# scaled_df = save_scalers_and_scale(encoded_df, continuous_columns, '/path/to/scalers_directory')

In [193]:
final_transformed_dataset = save_scalers_and_scale(
    df_encoded, cts_cols, '/Users/rammaruthi/Desktop/Truck/scalars/')

Scaler for column "route_avg_temp" saved to /Users/rammaruthi/Desktop/Truck/scalars/route_avg_temp_scaler.pkl
Scaler for column "route_avg_wind_speed" saved to /Users/rammaruthi/Desktop/Truck/scalars/route_avg_wind_speed_scaler.pkl
Scaler for column "route_avg_precip" saved to /Users/rammaruthi/Desktop/Truck/scalars/route_avg_precip_scaler.pkl
Scaler for column "route_avg_humidity" saved to /Users/rammaruthi/Desktop/Truck/scalars/route_avg_humidity_scaler.pkl
Scaler for column "route_avg_visibility" saved to /Users/rammaruthi/Desktop/Truck/scalars/route_avg_visibility_scaler.pkl
Scaler for column "route_avg_pressure" saved to /Users/rammaruthi/Desktop/Truck/scalars/route_avg_pressure_scaler.pkl
Scaler for column "distance" saved to /Users/rammaruthi/Desktop/Truck/scalars/distance_scaler.pkl
Scaler for column "average_hours" saved to /Users/rammaruthi/Desktop/Truck/scalars/average_hours_scaler.pkl
Scaler for column "origin_temp" saved to /Users/rammaruthi/Desktop/Truck/scalars/origin_te

In [194]:
df_encoded

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,gender_male,driving_style_proactive,ratings_3,ratings_4,ratings_5,ratings_6,ratings_7,ratings_8,ratings_9,is_midnight_1
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,-0.820579,0.647710,0.204021,1.492285,-0.279119,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,-0.184526,-0.862595,-0.468461,-0.843900,-0.279119,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,-0.876156,-0.093712,-1.040070,0.651259,-0.279119,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,-0.394485,0.894851,-1.712552,-1.703616,-0.279119,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,-1.357827,0.400569,0.170397,1.099806,-0.279119,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,1.038176,0.949771,-0.681413,-0.320594,-0.279119,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,-0.715599,-1.466716,-0.008932,-0.694384,6.637324,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,-0.357434,-1.411796,1.364052,0.034506,-0.279119,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,0.420649,-1.247036,0.798047,-0.619626,-0.279119,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [195]:
final_transformed_dataset

,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_humidity,route_avg_pressure,route_avg_precip,...,gender_male,driving_style_proactive,ratings_3,ratings_4,ratings_5,ratings_6,ratings_7,ratings_8,ratings_9,is_midnight_1
0,28411080,R-ef177918,2019-01-31 07:00:00,2019-01-31 19:24:36,0,-0.820579,0.647710,0.204021,1.492285,-0.279119,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,13193058,R-19a5b1a5,2019-01-26 07:00:00,2019-01-27 09:09:00,1,-0.184526,-0.862595,-0.468461,-0.843900,-0.279119,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,29077811,R-577e285c,2019-01-13 07:00:00,2019-01-13 10:42:36,0,-0.876156,-0.093712,-1.040070,0.651259,-0.279119,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12492275,R-a6df9ba2,2019-01-25 07:00:00,2019-01-25 15:17:23,0,-0.394485,0.894851,-1.712552,-1.703616,-0.279119,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,10784487,R-c970877c,2019-01-04 07:00:00,2019-01-04 11:22:12,0,-1.357827,0.400569,0.170397,1.099806,-0.279119,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10606,59301367,R-ebd8d425,2019-01-28 07:00:00,2019-01-28 13:16:48,0,1.038176,0.949771,-0.681413,-0.320594,-0.279119,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10607,58330582,R-04dfd423,2019-02-12 07:00:00,2019-02-12 15:24:00,1,-0.715599,-1.466716,-0.008932,-0.694384,6.637324,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10608,25670063,R-9d0d552b,2019-01-10 07:00:00,2019-01-10 23:21:36,1,-0.357434,-1.411796,1.364052,0.034506,-0.279119,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10609,42302347,R-269c91ef,2019-02-03 07:00:00,2019-02-03 17:22:48,0,0.420649,-1.247036,0.798047,-0.619626,-0.279119,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [196]:
final_transformed_dataset.dtypes

truck_id                      int32
route_id                     object
departure_date       datetime64[ns]
estimated_arrival    datetime64[ns]
delay                         int32
                          ...      
ratings_6                   float64
ratings_7                   float64
ratings_8                   float64
ratings_9                   float64
is_midnight_1               float64
Length: 164, dtype: object